In [11]:
import pandas as pd
import uuid
from datetime import datetime
import numpy as np
from grakn.client import GraknClient
from tqdm import tqdm
tqdm.pandas()

In [12]:
df=pd.read_excel(r"D:\Threat_intel_services\APT_collections\relation\mitigation\mitigation_relation.xlsx",sheet_name='tool_mitigation')

In [13]:
def  insert_general_info(df,ent_type,create):
    if create:
        for index,i in df.iterrows():
            df.internal_id_key[index]=str(uuid.uuid4())
            df.stix_id_key[index]=ent_type+'--'+str(uuid.uuid4())
            df.stix_label[index]='NA'
            df.entity_type[index]=ent_type
            df.created=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.created_at=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.created_at_day=(datetime.now()).strftime("%Y-%m-%d")
            df.created_at_month=(datetime.now()).strftime("%Y-%m")
            df.created_at_year=(datetime.now()).strftime("%Y")
            df.modified=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.updated_at=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.revoked=False
            df.object_status='NA'
    else:
            df.modified=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
            df.updated_at=(datetime.now()).strftime("%Y-%m-%dT%H:%M:%S")
    return df
    

In [14]:
tool_mit=insert_general_info(df=df,ent_type='relationship',create=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is t

In [46]:
def tool_mitigates_mitigation(df,session):
    graql_insert_query= f'match $t isa Tool,'\
                        f'has name \"'+df['tool']+'\";'\
                        f'$coa isa Course-Of-Action,'\
                        f'has name \"'+df['mitigation'].strip()+'\";'\
                        f'insert $uses (problem:$t ,mitigation:$coa) isa mitigates,'\
                        f"has internal_id_key \"{df['internal_id_key']}\", " \
                        f"has stix_id_key \"{df['stix_id_key']}\", " \
                        f"has entity_type \"{df['entity_type']}\", " \
                        f"has created {df['created']}, " \
                        f"has revoked \"{df['revoked']}\", "\
                        f"has created_at {df['created_at']}, " \
                        f"has created_at_day \"{df['created_at_day']}\", " \
                        f"has created_at_month \"{df['created_at_month']}\", " \
                        f"has created_at_year \"{df['created_at_year']}\", "\
                        f"has updated_at {df['updated_at']}; "
    print(graql_insert_query)
    try:
        with session.transaction().write() as write_transaction:
                # make a write transection with the query
                write_transaction.query(graql_insert_query)
                # remember to commit at the end
                write_transaction.commit()
    except:
        print("error occcured at _____________________________________"+df['mitigation'].strip())    
        pass

In [47]:
def load_data_into_grakn_relation(session,input_df):
    print("Inserting intrusion-set...")
    # using progress_apply instead of apply so we have a progress bar form tqdm
    # input_df.progress_apply(insert_one_intrusion_set, axis=1, session=session)
    print("Loading intrusion-set...")
    
    for index,row in input_df.iterrows():
        print(row['tool'])
        #intrusion_uses_malware(df=row,session=session)
        tool_mitigates_mitigation(df=row,session=session)

In [48]:
def build_grakn_graph_relation(input_df, keyspace_name):
    with GraknClient(uri="192.168.83.128:48555") as client:
        with client.session(keyspace = keyspace_name) as session:
            load_data_into_grakn_relation(session,input_df)

In [49]:
build_grakn_graph_relation(input_df=tool_mit,keyspace_name='grakn')

Inserting intrusion-set...
Loading intrusion-set...
HTRAN
match $t isa Tool,has name "HTRAN";$coa isa Course-Of-Action,has name "Network Intrusion Prevention";insert $uses (problem:$t ,mitigation:$coa) isa mitigates,has internal_id_key "6153113c-a649-42de-960a-78e646700e0a", has stix_id_key "relationship--c62a918e-0709-417f-8c58-f96b8c4e12cc", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has created_at_year "2020", has updated_at 2020-01-09T15:52:24; 
HTRAN
match $t isa Tool,has name "HTRAN";$coa isa Course-Of-Action,has name "Behavior Prevention on Endpoint";insert $uses (problem:$t ,mitigation:$coa) isa mitigates,has internal_id_key "d05a8008-e2aa-4ac5-bde4-a62b7d2f306e", has stix_id_key "relationship--a50535a3-d565-4ea4-b765-f25ea2b32aa0", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020

match $t isa Tool,has name "Koadic";$coa isa Course-Of-Action,has name "Privileged Process Integrity";insert $uses (problem:$t ,mitigation:$coa) isa mitigates,has internal_id_key "df4d78f8-e552-48e9-901d-fd5ba2b2d0fb", has stix_id_key "relationship--3550f3d1-4f26-4207-8c41-11d19aa4fa6f", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has created_at_year "2020", has updated_at 2020-01-09T15:52:24; 
Koadic
match $t isa Tool,has name "Koadic";$coa isa Course-Of-Action,has name "User Training";insert $uses (problem:$t ,mitigation:$coa) isa mitigates,has internal_id_key "7fa8c0a1-1da4-49c1-9af6-ca4561bb2148", has stix_id_key "relationship--e3163e5c-e6be-4928-9856-f01437396334", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2

match $t isa Tool,has name "Mimikatz";$coa isa Course-Of-Action,has name "Multi-factor Authentication";insert $uses (problem:$t ,mitigation:$coa) isa mitigates,has internal_id_key "dc1a5b2e-6744-4c35-bfd0-e8164cd06046", has stix_id_key "relationship--b124dedb-df92-4a3f-b430-2fab522ad471", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has created_at_year "2020", has updated_at 2020-01-09T15:52:24; 
Mimikatz
match $t isa Tool,has name "Mimikatz";$coa isa Course-Of-Action,has name "Network Segmentation";insert $uses (problem:$t ,mitigation:$coa) isa mitigates,has internal_id_key "f54e3d98-d0ee-4c2f-bd8c-f9e1055b761a", has stix_id_key "relationship--2e4dd120-4f98-4b7e-a0fe-bff59bab381a", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created

match $t isa Tool,has name "Mimikatz";$coa isa Course-Of-Action,has name "Active Directory Configuration";insert $uses (problem:$t ,mitigation:$coa) isa mitigates,has internal_id_key "82276769-c420-4c2b-95a6-6b8c7b720762", has stix_id_key "relationship--8751f510-7e9c-4571-93f7-30d2bc7bd8e2", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has created_at_year "2020", has updated_at 2020-01-09T15:52:24; 
Mimikatz
match $t isa Tool,has name "Mimikatz";$coa isa Course-Of-Action,has name "User Account Management";insert $uses (problem:$t ,mitigation:$coa) isa mitigates,has internal_id_key "86773304-3411-40ff-a788-20e444433454", has stix_id_key "relationship--f08114d4-21a0-4d30-bf0f-1c18329af127", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has c

match $t isa Tool,has name "Cobalt Strike";$coa isa Course-Of-Action,has name "Network Intrusion Prevention";insert $uses (problem:$t ,mitigation:$coa) isa mitigates,has internal_id_key "d7795501-c811-468c-8536-a3e9dab6bfe6", has stix_id_key "relationship--b0e334e5-9a9b-4990-b3ed-58d5d301743e", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has created_at_year "2020", has updated_at 2020-01-09T15:52:24; 
Cobalt Strike
match $t isa Tool,has name "Cobalt Strike";$coa isa Course-Of-Action,has name "Network Segmentation";insert $uses (problem:$t ,mitigation:$coa) isa mitigates,has internal_id_key "c47b8ae5-8dee-4790-a7fc-f24fd8d9c571", has stix_id_key "relationship--ac35f390-08b3-451c-8b6c-46b4b852e667", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-

match $t isa Tool,has name "Cobalt Strike";$coa isa Course-Of-Action,has name "Exploit Protection";insert $uses (problem:$t ,mitigation:$coa) isa mitigates,has internal_id_key "7d2c6a9b-82b4-4f18-b616-7de72c7b1065", has stix_id_key "relationship--6fdefbad-5a24-4a4b-8c97-1e5b80eb64fa", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has created_at_year "2020", has updated_at 2020-01-09T15:52:24; 
Cobalt Strike
match $t isa Tool,has name "Cobalt Strike";$coa isa Course-Of-Action,has name "Threat Intelligence Program";insert $uses (problem:$t ,mitigation:$coa) isa mitigates,has internal_id_key "05164e66-7d4a-43c6-85d0-de450a93ba08", has stix_id_key "relationship--4795be14-5990-4f28-82ee-a93beac8c0a9", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09"

match $t isa Tool,has name "Mimikatz";$coa isa Course-Of-Action,has name "Operating System Configuration";insert $uses (problem:$t ,mitigation:$coa) isa mitigates,has internal_id_key "9bc80162-1528-4a6b-ae46-95ece3a83767", has stix_id_key "relationship--f7a2ef8e-fdc6-40ef-b1ad-35b3018f7d8b", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has created_at_year "2020", has updated_at 2020-01-09T15:52:24; 
Mimikatz
match $t isa Tool,has name "Mimikatz";$coa isa Course-Of-Action,has name "Active Directory Configuration";insert $uses (problem:$t ,mitigation:$coa) isa mitigates,has internal_id_key "74cd7747-380b-4d00-ad44-9019cc7c555f", has stix_id_key "relationship--bc8bd1be-d92f-4a18-8656-4e7a241b2b8a", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09"

In [16]:
for index,df in tool_mit.iterrows():
    graql_insert_query= f'match $i isa Tool,'\
                        f'has name \"'+df['tool']+'\";'\
                        f'$m isa Course-Of-Action,'\
                        f'has name \"'+df['mitigation'].strip()+'\";'\
                        f'insert $uses (user:$i ,usage:$m) isa uses,'\
                        f"has internal_id_key \"{df['internal_id_key']}\", " \
                        f"has stix_id_key \"{df['stix_id_key']}\", " \
                        f"has entity_type \"{df['entity_type']}\", " \
                        f"has created {df['created']}, " \
                        f"has revoked \"{df['revoked']}\", "\
                        f"has created_at {df['created_at']}, " \
                        f"has created_at_day \"{df['created_at_day']}\", " \
                        f"has created_at_month \"{df['created_at_month']}\", " \
                        f"has created_at_year \"{df['created_at_year']}\", "\
                        f"has updated_at {df['updated_at']}; "
    print(graql_insert_query)

match $i isa Tool,has name "HTRAN";$m isa Course-Of-Action,has name "Network Intrusion Prevention";insert $uses (user:$i ,usage:$m) isa uses,has internal_id_key "6153113c-a649-42de-960a-78e646700e0a", has stix_id_key "relationship--c62a918e-0709-417f-8c58-f96b8c4e12cc", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has created_at_year "2020", has updated_at 2020-01-09T15:52:24; 
match $i isa Tool,has name "HTRAN";$m isa Course-Of-Action,has name "Behavior Prevention on Endpoint";insert $uses (user:$i ,usage:$m) isa uses,has internal_id_key "d05a8008-e2aa-4ac5-bde4-a62b7d2f306e", has stix_id_key "relationship--a50535a3-d565-4ea4-b765-f25ea2b32aa0", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has created_at_y

match $i isa Tool,has name "Koadic";$m isa Course-Of-Action,has name "User Training";insert $uses (user:$i ,usage:$m) isa uses,has internal_id_key "7fa8c0a1-1da4-49c1-9af6-ca4561bb2148", has stix_id_key "relationship--e3163e5c-e6be-4928-9856-f01437396334", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has created_at_year "2020", has updated_at 2020-01-09T15:52:24; 
match $i isa Tool,has name "Koadic";$m isa Course-Of-Action,has name "Disable or Remove Feature or Program";insert $uses (user:$i ,usage:$m) isa uses,has internal_id_key "3f560529-d23d-4ec6-a398-d9c63b33e892", has stix_id_key "relationship--61034bbb-dd94-45a3-b878-a6abc29b10ce", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has created_at_year "202

match $i isa Tool,has name "Mimikatz";$m isa Course-Of-Action,has name "Operating System Configuration";insert $uses (user:$i ,usage:$m) isa uses,has internal_id_key "3567c80e-0276-413c-9416-ddb2bbe1d80c", has stix_id_key "relationship--2bdbf96d-407d-4296-ac05-770bcbb60feb", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has created_at_year "2020", has updated_at 2020-01-09T15:52:24; 
match $i isa Tool,has name "Mimikatz";$m isa Course-Of-Action,has name "Privileged Account Management";insert $uses (user:$i ,usage:$m) isa uses,has internal_id_key "06088380-c71f-467a-a342-320483c19fc6", has stix_id_key "relationship--1cd967bc-6ab1-4896-8c92-a17ce8c120c7", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has create

match $i isa Tool,has name "Winexe";$m isa Course-Of-Action,has name "Privileged Account Management";insert $uses (user:$i ,usage:$m) isa uses,has internal_id_key "d929906d-7ba5-494c-820f-0bb8f97b754c", has stix_id_key "relationship--9310b595-93e3-400f-978f-8383e8d9a4cf", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has created_at_year "2020", has updated_at 2020-01-09T15:52:24; 
match $i isa Tool,has name "Winexe";$m isa Course-Of-Action,has name "Restrict File and Directory Permissions";insert $uses (user:$i ,usage:$m) isa uses,has internal_id_key "8c3b94d3-73ac-4a6a-9462-748b7d36a891", has stix_id_key "relationship--bd0d78f4-742f-4031-b1c1-ce0ccf0b28d7", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has c

match $i isa Tool,has name "Cobalt Strike";$m isa Course-Of-Action,has name "Network Intrusion Prevention";insert $uses (user:$i ,usage:$m) isa uses,has internal_id_key "6f1d233f-f829-41c0-b4c2-9e0bf13afdac", has stix_id_key "relationship--4f393cc7-413d-4bf7-bc8b-2c23693933ba", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has created_at_year "2020", has updated_at 2020-01-09T15:52:24; 
match $i isa Tool,has name "Cobalt Strike";$m isa Course-Of-Action,has name "Active Directory Configuration";insert $uses (user:$i ,usage:$m) isa uses,has internal_id_key "8e1771c8-5b40-48cf-957b-b002870e9672", has stix_id_key "relationship--ae7cb828-952c-469f-911a-e08217f6a24d", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", h

match $i isa Tool,has name "Cobalt Strike";$m isa Course-Of-Action,has name "Password Policies";insert $uses (user:$i ,usage:$m) isa uses,has internal_id_key "7a81d072-87a8-4b4a-b114-d2d1ef7d3eb9", has stix_id_key "relationship--0686c735-27b3-4b6f-89ed-f53350b8785c", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has created_at_year "2020", has updated_at 2020-01-09T15:52:24; 
match $i isa Tool,has name "Cobalt Strike";$m isa Course-Of-Action,has name "Privileged Account Management";insert $uses (user:$i ,usage:$m) isa uses,has internal_id_key "5e9e3f9c-fa9f-4916-9589-6ea0198e401d", has stix_id_key "relationship--98f2b965-c6b1-486e-b200-a09bf3bfa248", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has created_a

match $i isa Tool,has name "PsExec";$m isa Course-Of-Action,has name "Privileged Account Management";insert $uses (user:$i ,usage:$m) isa uses,has internal_id_key "0f09e7a5-43e6-41ef-ace5-2a1876560426", has stix_id_key "relationship--0c75e2d0-9cdd-478f-b38b-6bbbb4815449", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has created_at_year "2020", has updated_at 2020-01-09T15:52:24; 
match $i isa Tool,has name "PsExec";$m isa Course-Of-Action,has name "Restrict File and Directory Permissions";insert $uses (user:$i ,usage:$m) isa uses,has internal_id_key "6052a438-dd65-4f41-87d7-1baabbd0a562", has stix_id_key "relationship--6fc1d7e3-9e06-460a-89ab-cfc8c68990ad", has entity_type "relationship", has created 2020-01-09T15:52:24, has revoked "False", has created_at 2020-01-09T15:52:24, has created_at_day "2020-01-09", has created_at_month "2020-01", has c